# Max-Cut with QAOA (qsim / Cirq)

Simple QAOA Max-Cut example using Cirq and timing our search over
a small parameter grid.


In [ ]:
import time
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import cirq


In [ ]:
n_nodes = 6
G = nx.erdos_renyi_graph(n_nodes, 0.5, seed=1)
pos = nx.spring_layout(G, seed=1)
nx.draw(G, pos, with_labels=True)
plt.title("Max-Cut instance (qsim/Cirq)")
plt.show()


In [ ]:
qubits = [cirq.LineQubit(i) for i in range(len(G.nodes()))]

def qaoa_circuit(gamma, beta, reps=1):
    c = cirq.Circuit()
    # Init in |+> state
    for q in qubits:
        c.append(cirq.H(q))

    for _ in range(reps):
        # Cost layer
        for i, j in G.edges():
            c.append(cirq.ZZ(qubits[i], qubits[j]) ** (gamma / np.pi))
        # Mixer layer
        for q in qubits:
            c.append(cirq.X(q) ** (beta / np.pi))

    c.append(cirq.measure(*qubits, key="z"))
    return c

sim = cirq.Simulator()


In [ ]:
def maxcut_value(bitstring):
    # bitstring is int -> convert to binary string
    bstr = np.binary_repr(bitstring, width=len(qubits))
    bits = [int(b) for b in bstr]
    return sum(1 for i, j in G.edges() if bits[i] != bits[j])

def evaluate_params(gamma, beta, reps=1, shots=256):
    circuit = qaoa_circuit(gamma, beta, reps=reps)
    result = sim.run(circuit, repetitions=shots)
    counts = result.histogram(key="z")
    best_cut = 0
    for bitstr, c in counts.items():
        val = maxcut_value(bitstr)
        if val > best_cut:
            best_cut = val
    return best_cut


In [ ]:
start = time.perf_counter()

best_cut = -1
best_params = None
gammas = np.linspace(0, np.pi, 7)
betas = np.linspace(0, np.pi, 7)

for g in gammas:
    for b in betas:
        val = evaluate_params(g, b, reps=1, shots=256)
        if val > best_cut:
            best_cut = val
            best_params = (g, b)

end = time.perf_counter()

print("Best cut value:", best_cut)
print("Best params (gamma, beta):", best_params)
print(f"Runtime (grid search): {end - start:.4f} s")
